In [ ]:
import sys, os, time
from datetime import datetime, date, timedelta
import pandas as pd
pd.set_option('mode.chained_assignment',None)
import pickle
import json
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from scipy import stats
from scipy.stats import norm
import scipy.optimize as optimize
from scipy.optimize import minimize
from scipy.interpolate import UnivariateSpline
import math
import copy
import shelve
import itertools
from sklearn import linear_model
from __future__ import absolute_import, division, print_function, unicode_literals

In [ ]:
try:
  %tensorflow_version 2.x # %tensorflow_version only exists in Colab.
except Exception:
  pass

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x # %tensorflow_version only exists in Colab.`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
tf.keras.backend.clear_session()  # For easy reset of notebook state
tf.keras.backend.set_floatx('float32')
print(tf.__version__)

2.6.0


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K

In [ ]:
from google.colab import drive # import google drive functionality from google colab
drive.mount('/content/drive/',force_remount=True) # mount drive with inital path '/content/drive/'

Mounted at /content/drive/


In [1]:
#@markdown path variable should be changed as appropriate depending on what path to the project you have configured.
path = "drive/My Drive/dev/sdi/" #@param {type:"string"}

# (1) Get World Bank datasets

In [ ]:
gdp_df = pd.read_csv(path + 'data/datasets/gdp.csv')
gdp_df

,iso3,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AFG,1.780429e+10,2.000160e+10,2.056107e+10,2.048489e+10,1.990711e+10,1.801775e+10,1.886995e+10,1.835388e+10,1.929110e+10,1.980707e+10
1,AGO,1.117897e+11,1.280529e+11,1.367099e+11,1.457122e+11,1.161936e+11,1.011239e+11,1.221238e+11,1.013532e+11,8.941719e+10,6.230691e+10
2,ALB,1.289077e+10,1.231983e+10,1.277622e+10,1.322814e+10,1.138685e+10,1.186120e+10,1.301969e+10,1.514702e+10,1.528661e+10,1.479962e+10
3,AND,3.629204e+09,3.188809e+09,3.193704e+09,3.271808e+09,2.789870e+09,2.896679e+09,3.000181e+09,3.218316e+09,3.155065e+09,NaN
4,ARE,3.506660e+11,3.745906e+11,3.901076e+11,4.031371e+11,3.581351e+11,3.570451e+11,3.856055e+11,4.222150e+11,4.211423e+11,NaN
...,...,...,...,...,...,...,...,...,...,...,...
194,WSM,7.374017e+08,7.605496e+08,7.700217e+08,7.569199e+08,7.883073e+08,7.993764e+08,8.321536e+08,8.214961e+08,8.522502e+08,8.070272e+08
195,XKX,6.701698e+09,6.499807e+09,7.074778e+09,7.396705e+09,6.442916e+09,6.719172e+09,7.245707e+09,7.942962e+09,7.953156e+09,7.611402e+09
196,ZAF,4.164189e+11,3.963327e+11,3.668294e+11,3.509046e+11,3.176205e+11,2.963573e+11,3.495541e+11,3.682889e+11,3.514316e+11,3.019236e+11
197,ZMB,2.345952e+10,2.550306e+10,2.803724e+10,2.714102e+10,2.125122e+10,2.095841e+10,2.587360e+10,2.631164e+10,2.330869e+10,1.932005e+10


In [ ]:
gdp = {'fdi':False,
       'dci':False,
       'trsb':False,
       'sis':False,
       'ttcr':False,
       'eif':False,
       'bedi':False,
       'dcps':False,
       'slri':False,
       'ans':False}
files = {'fdi':'API_BX.KLT.DINV.WD.GD.ZS_DS2_en_excel_v2_3159499.xls',
         'dci':'API_IC.CRD.INFO.XQ_DS2_en_excel_v2_3054127.xls',
         'trsb':'API_IC.REG.DURS_DS2_en_excel_v2_3058786.xls',
         'sis':'API_IT.NET.SECR.P6_DS2_en_excel_v2_3170414.xls',
         'ttcr':'API_IC.TAX.TOTL.CP.ZS_DS2_en_excel_v2_3169344.xls',
         'eif':'API_SL.IND.EMPL.FE.ZS_DS2_en_excel_v2_3168324.xls',
         'bedi':'API_IC.BUS.DISC.XQ_DS2_en_excel_v2_3165884.xls',
         'dcps':'API_FS.AST.PRVT.GD.ZS_DS2_en_excel_v2_3052622.xls',
         'slri':'API_IC.LGL.CRED.XQ_DS2_en_excel_v2_3054136.xls',
         'ans':'API_NY.ADJ.SVNG.GN.ZS_DS2_en_excel_v2_3053043.xls'}
quantiles = {'low':0.5,'high':1.}
columns = ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

In [ ]:
worldbank_links_df = gdp_df.copy()
worldbank_links_df = worldbank_links_df['iso3']
worldbank_links_df = worldbank_links_df.to_frame()
worldbank_links_df

,iso3
0,AFG
1,AGO
2,ALB
3,AND
4,ARE
...,...
194,WSM
195,XKX
196,ZAF
197,ZMB


In [ ]:
for big_key in files: # Iterate through files disctionary to preprocess each file
  spreadsheet_df = pd.read_excel(path+'data/worldbank/'+big_key+'/'+files[big_key],skiprows=3,sheet_name='Data') # load in spreadhseet of iteration as dataframe
  gdp_iteration_df = gdp_df.copy() # copy GDP dataframe to a new variable
  gdp_iteration_df['mean'] = None # create 'mean' column with no values
  for i in gdp_iteration_df.index.tolist(): # Loop through each row that is in gdp_iteration_df
    narrowed_countries_df = spreadsheet_df['Country Code']==gdp_iteration_df['iso3'].loc[i] # select 1 country
    if gdp[big_key]: # If we want to divide by gdp
      scaled_df = spreadsheet_df[columns][narrowed_countries_df].values[0]/gdp_iteration_df[columns].loc[i].values # divide last 10 years of data in main spreadsheet of one country by last 10 years of GDP of that country
    elif big_key == 'trsb': # if we are in the 'time required to start a business' loop
      scaled_df = [1/x for x in spreadsheet_df[columns][narrowed_countries_df].values[0]] # take reciprocals of values
    else: # If we don't need to divide by GDP
      scaled_df = spreadsheet_df[columns][narrowed_countries_df].copy() # copy dataframe of the country data into scaled_df
    gdp_iteration_df['mean'].loc[i] = np.nanmean(scaled_df) # take the mean of the data of the country into the gdp_iteration_df['mean'] column
  gdp_iteration_df = gdp_iteration_df[['iso3','mean']] # removing other columns and having only country and mean columns
  gdp_iteration_df[big_key + '_mean'] = gdp_iteration_df['mean'] # renaming the means of all the countries to the dataset abbreviation mean
  gdp_iteration_df[big_key] = None # creating empty column called the dataset abbreviation
  na_df = gdp_iteration_df['mean'].isna() # select rows with NaN value in 'mean' column and call it na_df
  gdp_iteration_df[big_key][na_df] = 'unknown' # replace the NaN value with the name of the folder and '_unknown'
  q = 0.
  for key in quantiles: # looping through quantiles dictionary where key is 'low', 'medium', 'high' or 'highest based on iteration
    lower_cutoff = gdp_iteration_df['mean'].quantile(q) # selecting number thats in the qth percentile of the data
    upper_cutoff = gdp_iteration_df['mean'].quantile(quantiles[key]) # selecting number thats in the quantiles[key]th percentile (0.25 higher than q)
    one_quarter = (gdp_iteration_df['mean']>=lower_cutoff)&(gdp_iteration_df['mean']<upper_cutoff) # selecting rows that are inbetween lower_cutoff and upper_cutoff
    gdp_iteration_df[big_key][one_quarter] = key
    q = quantiles[key] # setting new q up 0.25 (25%)
  worldbank_links_df = pd.concat([worldbank_links_df,gdp_iteration_df[[big_key, big_key + '_mean']]],axis=1)

worldbank_links_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: Mean of empty slice
  del sys.path[0]


,iso3,fdi,fdi_mean,dci,dci_mean,trsb,trsb_mean,sis,sis_mean,ttcr,ttcr_mean,eif,eif_mean,bedi,bedi_mean,dcps,dcps_mean,slri,slri_mean,ans,ans_mean
0,AFG,low,0.381796,low,0,high,0.118917,low,17.7895,high,49.0556,high,21.79,low,2.55556,low,3.92743,high,9.28571,unknown,NaN
1,AGO,low,-1.67825,low,0,low,0.0221661,low,8.8834,high,50.4556,low,1.01222,low,4,low,17.4403,low,1,low,0.0921652
2,ALB,high,8.36168,high,6,high,0.201235,high,295.592,low,36.2,high,13.3322,high,8.11111,low,37.5782,high,7,low,1.55205
3,AND,unknown,NaN,unknown,NaN,unknown,NaN,high,4750.72,unknown,NaN,unknown,NaN,unknown,NaN,unknown,NaN,unknown,NaN,unknown,NaN
4,ARE,low,2.59326,high,7.28571,high,0.147545,high,861.708,low,15.3444,low,6.01222,high,8.22222,high,81.0307,low,3.14286,unknown,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,WSM,low,1.86616,low,0,high,0.111111,high,234.871,low,18.6444,high,37.4367,low,5,high,78.7057,high,6.71429,unknown,NaN
195,XKX,high,4.55127,high,6,high,0.113762,unknown,NaN,low,15.3556,unknown,NaN,high,6,low,38.8465,high,9.85714,unknown,NaN
196,ZAF,low,1.20437,high,7.14286,low,0.0227322,high,5204.61,low,29.5778,high,12.12,high,8,high,135.523,low,5,low,0.628886
197,ZMB,high,4.817,high,7.57143,high,0.103357,low,18.4873,low,16.2222,low,4.13222,low,4,low,15.9078,high,8.71429,high,17.033


In [ ]:
# Randomize unknowns to complete the data
np.random.seed(0)
replace_nas = ['low','high']
x = worldbank_links_df.copy()
for d in gdp:
  k = x[d]=='unknown'
  if any(k):
    x[d][k] = np.random.choice(replace_nas,sum(k))
x = x[['iso3']+list(gdp.keys())]
x

,iso3,fdi,dci,trsb,sis,ttcr,eif,bedi,dcps,slri,ans
0,AFG,low,low,high,low,high,high,low,low,high,high
1,AGO,low,low,low,low,high,low,low,low,low,low
2,ALB,high,high,high,high,low,high,high,low,high,low
3,AND,low,high,low,high,low,high,high,low,high,high
4,ARE,low,high,high,high,low,low,high,high,low,low
...,...,...,...,...,...,...,...,...,...,...,...
194,WSM,low,low,high,high,low,high,low,high,high,low
195,XKX,high,high,high,low,low,low,high,low,high,low
196,ZAF,low,high,low,high,low,high,high,high,low,low
197,ZMB,high,high,high,low,low,low,low,low,high,high


In [ ]:
x.to_pickle(path+'data/datasets/wb_dataframe.pkl')

# (2) Get SDGs data

In [ ]:
sdg_index_df = pd.read_excel(path+'data/sdgindex.org/SDR 2021 - Database.xlsx',sheet_name='SDR2021 Data') # Read 'SDR2021 Data' tab of 'SDR 2021 - Database.xlsx' in
non_country = sdg_index_df['Country Code ISO3'].str.len()==3 
sdg_index_df = sdg_index_df[non_country].reset_index(drop=True) # Drop non country rows
sdg_index_df

,Country Code ISO3,Country,2021 SDG Index Score,2021 SDG Index Rank,Percentage missing values,Spillover Score (0-100),Regional Score (0-100),Regions used for the SDR,Population in 2020,Goal 1 Dash,Goal 1 Trend,Goal 2 Dash,Goal 2 Trend,Goal 3 Dash,Goal 3 Trend,Goal 4 Dash,Goal 4 Trend,Goal 5 Dash,Goal 5 Trend,Goal 6 Dash,Goal 6 Trend,Goal 7 Dash,Goal 7 Trend,Goal 8 Dash,Goal 8 Trend,Goal 9 Dash,Goal 9 Trend,Goal 10 Dash,Goal 10 Trend,Goal 11 Dash,Goal 11 Trend,Goal 12 Dash,Goal 12 Trend,Goal 13 Dash,Goal 13 Trend,Goal 14 Dash,Goal 14 Trend,Goal 15 Dash,Goal 15 Trend,Goal 16 Dash,...,Dashboard Color: sdg17_sprofits,Statistical Performance Index (worst 0-100 best),Year: sdg17_statperf,Normalized Score: sdg17_statperf,Dashboard Color: sdg17_statperf,Trend: sdg17_statperf,Goal 1 Score,Goal 2 Score,Goal 3 Score,Goal 4 Score,Goal 5 Score,Goal 6 Score,Goal 7 Score,Goal 8 Score,Goal 9 Score,Goal 10 Score,Goal 11 Score,Goal 12 Score,Goal 13 Score,Goal 14 Score,Goal 15 Score,Goal 16 Score,Goal 17 Score,Goal 1 Regional Score,Goal 2 Regional Score,Goal 3 Regional Score,Goal 4 Regional Score,Goal 5 Regional Score,Goal 6 Regional Score,Goal 7 Regional Score,Goal 8 Regional Score,Goal 9 Regional Score,Goal 10 Regional Score,Goal 11 Regional Score,Goal 12 Regional Score,Goal 13 Regional Score,Goal 14 Regional Score,Goal 15 Regional Score,Goal 16 Regional Score,Goal 17 Regional Score
0,AFG,Afghanistan,53.933312,137.0,9.302326,99.344000,71.402435,E. Europe & C. Asia,38928341.0,grey,NaN,red,→,red,→,red,➚,red,→,red,➚,orange,➚,red,➚,red,→,grey,NaN,red,→,yellow,NaN,green,↑,grey,NaN,red,↓,red,...,NaN,49.756,2019.0,33.008,red,↑,NaN,50.131000,37.098357,57.851217,23.16250,46.3138,64.822500,37.753167,6.546833,NaN,31.761250,90.816833,99.064667,NaN,52.9840,52.026667,43.359500,91.945432,61.734196,77.282636,91.682781,60.848884,71.824574,87.673500,69.774644,40.058590,72.362762,76.443228,77.629345,83.041913,58.865821,69.499564,72.403938,63.386460
1,AGO,Angola,50.296134,154.0,0.000000,97.022200,51.926818,Africa,32866268.0,red,↓,red,→,red,→,red,NaN,red,→,red,➚,red,➚,red,➚,red,→,red,NaN,red,→,green,NaN,yellow,↑,orange,→,red,→,red,...,NaN,54.946,2019.0,39.928,orange,↑,12.9130,55.424125,37.443571,40.775667,51.27800,53.9610,52.432667,52.485500,7.517000,25.2465,46.711750,92.220167,96.520333,66.9320,64.7534,50.653600,47.766000,30.344163,51.977747,44.140162,53.624857,53.011210,52.579836,37.276583,59.981185,18.239817,37.334205,52.728915,88.024702,96.266527,68.010492,66.835003,56.050983,51.652955
2,ALB,Albania,71.021858,64.0,3.296703,94.337556,71.402435,E. Europe & C. Asia,2877800.0,yellow,➚,orange,→,orange,➚,green,↑,orange,→,orange,➚,yellow,↑,red,➚,orange,➚,orange,NaN,orange,→,orange,NaN,green,↑,red,→,orange,→,orange,...,NaN,75.383,2019.0,67.177,yellow,↑,91.6375,60.004857,81.092714,95.155000,53.57425,75.9076,89.449667,61.095333,28.580500,68.4005,72.838333,79.617333,92.126000,43.8784,80.5868,68.091800,65.335000,91.945432,61.734196,77.282636,91.682781,60.848884,71.824574,87.673500,69.774644,40.058590,72.362762,76.443228,77.629345,83.041913,58.865821,69.499564,72.403938,63.386460
3,AND,Andorra,NaN,NaN,51.162791,NaN,71.402435,E. Europe & C. Asia,77265.0,grey,NaN,yellow,➚,grey,NaN,grey,NaN,green,↑,yellow,➚,green,↑,grey,NaN,orange,➚,grey,NaN,yellow,↑,orange,NaN,red,→,grey,NaN,orange,➚,yellow,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ARE,United Arab Emirates,70.169932,71.0,10.989011,38.819636,67.100057,MENA,9890400.0,green,↑,red,→,yellow,↑,yellow,↑,orange,➚,red,➚,yellow,➚,red,➚,yellow,↑,yellow,NaN,yellow,➚,red,NaN,red,➚,red,➚,grey,NaN,orange,...,NaN,59.669,2019.0,46.225,orange,↑,99.5695,67.268000,86.388077,92.059333,71.19925,56.1116,90.990667,60.130600,74.105167,91.3750,75.975500,51.127500,29.150000,76.6002,58.6250,79.212444,33.001000,92.321500,59.184666,75.868532,84.106553,44.589750,59.044700,88.106208,62.065689,47.493910,66.178091,58.928271,76.842146,71

In [ ]:
columns = ['Country Code ISO3','Country','Regions used for the SDR','Population in 2020']
for i in range(17): # Adding 'Goal NUMBER Dash' and 'Goal NUMBER Trend' strings to columns list
  columns = columns+['Goal '+str(i+1)+' Dash','Goal '+str(i+1)+' Trend']
sdg_index_df = sdg_index_df[columns] # Drop columns except the ones in the columns list
sdg_index_df

,Country Code ISO3,Country,Regions used for the SDR,Population in 2020,Goal 1 Dash,Goal 1 Trend,Goal 2 Dash,Goal 2 Trend,Goal 3 Dash,Goal 3 Trend,Goal 4 Dash,Goal 4 Trend,Goal 5 Dash,Goal 5 Trend,Goal 6 Dash,Goal 6 Trend,Goal 7 Dash,Goal 7 Trend,Goal 8 Dash,Goal 8 Trend,Goal 9 Dash,Goal 9 Trend,Goal 10 Dash,Goal 10 Trend,Goal 11 Dash,Goal 11 Trend,Goal 12 Dash,Goal 12 Trend,Goal 13 Dash,Goal 13 Trend,Goal 14 Dash,Goal 14 Trend,Goal 15 Dash,Goal 15 Trend,Goal 16 Dash,Goal 16 Trend,Goal 17 Dash,Goal 17 Trend
0,AFG,Afghanistan,E. Europe & C. Asia,38928341.0,grey,NaN,red,→,red,→,red,➚,red,→,red,➚,orange,➚,red,➚,red,→,grey,NaN,red,→,yellow,NaN,green,↑,grey,NaN,red,↓,red,→,red,➚
1,AGO,Angola,Africa,32866268.0,red,↓,red,→,red,→,red,NaN,red,→,red,➚,red,➚,red,➚,red,→,red,NaN,red,→,green,NaN,yellow,↑,orange,→,red,→,red,➚,orange,➚
2,ALB,Albania,E. Europe & C. Asia,2877800.0,yellow,➚,orange,→,orange,➚,green,↑,orange,→,orange,➚,yellow,↑,red,➚,orange,➚,orange,NaN,orange,→,orange,NaN,green,↑,red,→,orange,→,orange,→,orange,➚
3,AND,Andorra,E. Europe & C. Asia,77265.0,grey,NaN,yellow,➚,grey,NaN,grey,NaN,green,↑,yellow,➚,green,↑,grey,NaN,orange,➚,grey,NaN,yellow,↑,orange,NaN,red,→,grey,NaN,orange,➚,yellow,↑,grey,NaN
4,ARE,United Arab Emirates,MENA,9890400.0,green,↑,red,→,yellow,↑,yellow,↑,orange,➚,red,➚,yellow,➚,red,➚,yellow,↑,yellow,NaN,yellow,➚,red,NaN,red,➚,red,➚,grey,NaN,orange,➚,orange,↑
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,WSM,Samoa,Oceania,198410.0,yellow,➚,red,→,red,➚,green,↑,red,→,orange,↑,yellow,➚,orange,→,red,→,red,NaN,yellow,➚,red,NaN,green,↑,orange,→,red,→,orange,↑,orange,→
189,YEM,"Yemen, Rep.",MENA,29825968.0,grey,NaN,red,→,red,→,red,NaN,red,→,red,➚,red,↓,red,→,red,→,red,NaN,red,→,yellow,NaN,green,↑,red,→,red,↓,red,→,orange,↓
190,ZAF,South Africa,Africa,59308690.0,red,↓,red,→,red,➚,orange,↓,yellow,↑,orange,➚,orange,➚,red,→,orange,➚,red,NaN,orange,➚,yellow,NaN,orange,➚,red,→,red,→,red,→,yellow,➚
191,ZMB,Zambia,Africa,18383956.0,red,↓,red,→,red,➚,orange,NaN,orange,→,red,→,red,➚,red,➚,red,→,red,NaN,red,→,green,NaN,green,↑,grey,NaN,red,→,red,→,orange,→


In [ ]:
columns = ['iso3','country','region','population'] # Making renamed column list
for i in range(17): # adding 'NUMBER_dash' and 'NUMBER_trend' strings to column list
  columns = columns+[str(i+1)+'_goal',str(i+1)+'_trend']
sdg_index_df.columns = columns # Replacing column names with strings in column list
sdg_index_df

,iso3,country,region,population,1_goal,1_trend,2_goal,2_trend,3_goal,3_trend,4_goal,4_trend,5_goal,5_trend,6_goal,6_trend,7_goal,7_trend,8_goal,8_trend,9_goal,9_trend,10_goal,10_trend,11_goal,11_trend,12_goal,12_trend,13_goal,13_trend,14_goal,14_trend,15_goal,15_trend,16_goal,16_trend,17_goal,17_trend
0,AFG,Afghanistan,E. Europe & C. Asia,38928341.0,grey,NaN,red,→,red,→,red,➚,red,→,red,➚,orange,➚,red,➚,red,→,grey,NaN,red,→,yellow,NaN,green,↑,grey,NaN,red,↓,red,→,red,➚
1,AGO,Angola,Africa,32866268.0,red,↓,red,→,red,→,red,NaN,red,→,red,➚,red,➚,red,➚,red,→,red,NaN,red,→,green,NaN,yellow,↑,orange,→,red,→,red,➚,orange,➚
2,ALB,Albania,E. Europe & C. Asia,2877800.0,yellow,➚,orange,→,orange,➚,green,↑,orange,→,orange,➚,yellow,↑,red,➚,orange,➚,orange,NaN,orange,→,orange,NaN,green,↑,red,→,orange,→,orange,→,orange,➚
3,AND,Andorra,E. Europe & C. Asia,77265.0,grey,NaN,yellow,➚,grey,NaN,grey,NaN,green,↑,yellow,➚,green,↑,grey,NaN,orange,➚,grey,NaN,yellow,↑,orange,NaN,red,→,grey,NaN,orange,➚,yellow,↑,grey,NaN
4,ARE,United Arab Emirates,MENA,9890400.0,green,↑,red,→,yellow,↑,yellow,↑,orange,➚,red,➚,yellow,➚,red,➚,yellow,↑,yellow,NaN,yellow,➚,red,NaN,red,➚,red,➚,grey,NaN,orange,➚,orange,↑
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,WSM,Samoa,Oceania,198410.0,yellow,➚,red,→,red,➚,green,↑,red,→,orange,↑,yellow,➚,orange,→,red,→,red,NaN,yellow,➚,red,NaN,green,↑,orange,→,red,→,orange,↑,orange,→
189,YEM,"Yemen, Rep.",MENA,29825968.0,grey,NaN,red,→,red,→,red,NaN,red,→,red,➚,red,↓,red,→,red,→,red,NaN,red,→,yellow,NaN,green,↑,red,→,red,↓,red,→,orange,↓
190,ZAF,South Africa,Africa,59308690.0,red,↓,red,→,red,➚,orange,↓,yellow,↑,orange,➚,orange,➚,red,→,orange,➚,red,NaN,orange,➚,yellow,NaN,orange,➚,red,→,red,→,red,→,yellow,➚
191,ZMB,Zambia,Africa,18383956.0,red,↓,red,→,red,➚,orange,NaN,orange,→,red,→,red,➚,red,➚,red,→,red,NaN,red,→,green,NaN,green,↑,grey,NaN,red,→,red,→,orange,→


In [ ]:
trend = {'↓':'decreasing',
         '→':'stagnating',
         '➚':'increasing',
         '↑':'ontrack'} # Making disctionary associating trend arrows with words
for i in range(17): # Loop to replace characters in 'NUMBER_trend' columns with words as associated in dictionary
  column = str(i+1)+'_trend' # Selecting column for iteration
  k = sdg_index_df[column].isna() # selecting all rows with NaN in cell in column of the iteration
  sdg_index_df[column][k] = 'unknown' # Replacing selected NaN values with 'unknown'
  for key in trend:
    k = sdg_index_df[column]==key
    if any(k):
      sdg_index_df[column][k] = trend[key]
sdg_index_df

,iso3,country,region,population,1_goal,1_trend,2_goal,2_trend,3_goal,3_trend,4_goal,4_trend,5_goal,5_trend,6_goal,6_trend,7_goal,7_trend,8_goal,8_trend,9_goal,9_trend,10_goal,10_trend,11_goal,11_trend,12_goal,12_trend,13_goal,13_trend,14_goal,14_trend,15_goal,15_trend,16_goal,16_trend,17_goal,17_trend
0,AFG,Afghanistan,E. Europe & C. Asia,38928341.0,grey,unknown,red,stagnating,red,stagnating,red,increasing,red,stagnating,red,increasing,orange,increasing,red,increasing,red,stagnating,grey,unknown,red,stagnating,yellow,unknown,green,ontrack,grey,unknown,red,decreasing,red,stagnating,red,increasing
1,AGO,Angola,Africa,32866268.0,red,decreasing,red,stagnating,red,stagnating,red,unknown,red,stagnating,red,increasing,red,increasing,red,increasing,red,stagnating,red,unknown,red,stagnating,green,unknown,yellow,ontrack,orange,stagnating,red,stagnating,red,increasing,orange,increasing
2,ALB,Albania,E. Europe & C. Asia,2877800.0,yellow,increasing,orange,stagnating,orange,increasing,green,ontrack,orange,stagnating,orange,increasing,yellow,ontrack,red,increasing,orange,increasing,orange,unknown,orange,stagnating,orange,unknown,green,ontrack,red,stagnating,orange,stagnating,orange,stagnating,orange,increasing
3,AND,Andorra,E. Europe & C. Asia,77265.0,grey,unknown,yellow,increasing,grey,unknown,grey,unknown,green,ontrack,yellow,increasing,green,ontrack,grey,unknown,orange,increasing,grey,unknown,yellow,ontrack,orange,unknown,red,stagnating,grey,unknown,orange,increasing,yellow,ontrack,grey,unknown
4,ARE,United Arab Emirates,MENA,9890400.0,green,ontrack,red,stagnating,yellow,ontrack,yellow,ontrack,orange,increasing,red,increasing,yellow,increasing,red,increasing,yellow,ontrack,yellow,unknown,yellow,increasing,red,unknown,red,increasing,red,increasing,grey,unknown,orange,increasing,orange,ontrack
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,WSM,Samoa,Oceania,198410.0,yellow,increasing,red,stagnating,red,increasing,green,ontrack,red,stagnating,orange,ontrack,yellow,increasing,orange,stagnating,red,stagnating,red,unknown,yellow,increasing,red,unknown,green,ontrack,orange,stagnating,red,stagnating,orange,ontrack,orange,stagnating
189,YEM,"Yemen, Rep.",MENA,29825968.0,grey,unknown,red,stagnating,red,stagnating,red,unknown,red,stagnating,red,increasing,red,decreasing,red,stagnating,red,stagnating,red,unknown,red,stagnating,yellow,unknown,green,ontrack,red,stagnating,red,decreasing,red,stagnating,orange,decreasing
190,ZAF,South Africa,Africa,59308690.0,red,decreasing,red,stagnating,red,increasing,orange,decreasing,yellow,ontrack,orange,increasing,orange,increasing,red,stagnating,orange,increasing,red,unknown,orange,increasing,yellow,unknown,orange,increasing,red,stagnating,red,stagnating,red,stagnating,yellow,increasing
191,ZMB,Zambia,Africa,18383956.0,red,decreasing,red,stagnating,red,increasing,orange,unknown,orange,stagnating,red,stagnating,red,increasing,red,increasing,red,stagnating,red,unknown,red,stagnating,green,unknown,green,ontrack,grey,unknown,red,stagnating,red,stagnating,orange,stagnating


In [ ]:
# Testing if there are NAs.
for i in range(17):
  column = str(i+1)+'_goal'
  k = sdg_index_df[column].isna()
  if any(k):
    print(column)

In [ ]:
scores = {'grey':np.nan,'red':0.25,'orange':0.5,'yellow':0.75,'green':1.}
replace_nas = ['red','orange','yellow','green']
sdgs = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17']

In [ ]:
# Randomize unknowns to complete the data
np.random.seed(0)
y = sdg_index_df.copy()
for i in y.index.tolist():
  for s in sdgs: 
    y.rename(columns={s+'_goal':s},inplace=True)
    for r in scores:
      k = y[s]==r
      if any(k):
        y[s][k] = scores[r]
    k = y[s].isna()
    if any(k):
      y[s][k] = np.random.choice(replace_nas,sum(k))
y = y[['iso3']+sdgs]
y

,iso3,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,AFG,0.25,0.25,0.25,0.25,0.25,0.25,0.5,0.25,0.25,0.5,0.25,0.75,1,0.25,0.25,0.25,0.25
1,AGO,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,1,0.75,0.5,0.25,0.25,0.5
2,ALB,0.75,0.5,0.5,1,0.5,0.5,0.75,0.25,0.5,0.5,0.5,0.5,1,0.25,0.5,0.5,0.5
3,AND,1,0.75,0.75,1,1,0.75,1,1,0.5,0.75,0.75,0.5,0.25,1,0.5,0.75,0.25
4,ARE,1,0.25,0.75,0.75,0.5,0.25,0.75,0.25,0.75,0.75,0.75,0.25,0.25,0.25,1,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,WSM,0.75,0.25,0.25,1,0.25,0.5,0.75,0.5,0.25,0.25,0.75,0.25,1,0.5,0.25,0.5,0.5
189,YEM,0.5,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.75,1,0.25,0.25,0.25,0.5
190,ZAF,0.25,0.25,0.25,0.5,0.75,0.5,0.5,0.25,0.5,0.25,0.5,0.75,0.5,0.25,0.25,0.25,0.75
191,ZMB,0.25,0.25,0.25,0.5,0.5,0.25,0.25,0.25,0.25,0.25,0.25,1,1,0.25,0.25,0.25,0.5


In [ ]:
y.to_pickle(path+'data/datasets/sdg_dataframe.pkl')

# (3) Create train dataset

In [ ]:
x = pd.read_pickle(path+'data/datasets/wb_dataframe.pkl')
y = pd.read_pickle(path+'data/datasets/sdg_dataframe.pkl')

In [ ]:
# Filter out WB countries not present in SDG dataset
k = x['iso3'].isin(y['iso3'])
x = x[k].sort_values(by=['iso3']).reset_index(drop=True)
x

,iso3,fdi,dci,trsb,sis,ttcr,eif,bedi,dcps,slri,ans
0,AFG,low,low,high,low,high,high,low,low,high,high
1,AGO,low,low,low,low,high,low,low,low,low,low
2,ALB,high,high,high,high,low,high,high,low,high,low
3,AND,low,high,low,high,low,high,high,low,high,high
4,ARE,low,high,high,high,low,low,high,high,low,low
...,...,...,...,...,...,...,...,...,...,...,...
181,VUT,high,low,low,low,low,low,low,high,high,high
182,WSM,low,low,high,high,low,high,low,high,high,low
183,ZAF,low,high,low,high,low,high,high,high,low,low
184,ZMB,high,high,high,low,low,low,low,low,high,high


In [ ]:
# Generate one-hot encoded vectors for WB datasets
datasets = ['fdi','dci','trsb','sis','ttcr','eif','bedi','dcps','slri','ans']
x['input'] = None
for i in x.index.tolist():
  vector = []
  for d in datasets:
    if x[d].loc[i]=='low':
      vector = vector+[0.]
    else:
      vector = vector+[1.]
  x['input'].loc[i] = vector
x

,iso3,fdi,dci,trsb,sis,ttcr,eif,bedi,dcps,slri,ans,input
0,AFG,low,low,high,low,high,high,low,low,high,high,"[0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, ..."
1,AGO,low,low,low,low,high,low,low,low,low,low,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
2,ALB,high,high,high,high,low,high,high,low,high,low,"[1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, ..."
3,AND,low,high,low,high,low,high,high,low,high,high,"[0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, ..."
4,ARE,low,high,high,high,low,low,high,high,low,low,"[0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
181,VUT,high,low,low,low,low,low,low,high,high,high,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, ..."
182,WSM,low,low,high,high,low,high,low,high,high,low,"[0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, ..."
183,ZAF,low,high,low,high,low,high,high,high,low,low,"[0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, ..."
184,ZMB,high,high,high,low,low,low,low,low,high,high,"[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."


In [ ]:
# Filter out SDG countries not present in WB dataset
k = y['iso3'].isin(x['iso3'])
y = y[k].sort_values(by=['iso3']).reset_index(drop=True)
y

,iso3,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,AFG,0.25,0.25,0.25,0.25,0.25,0.25,0.5,0.25,0.25,0.5,0.25,0.75,1,0.25,0.25,0.25,0.25
1,AGO,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,1,0.75,0.5,0.25,0.25,0.5
2,ALB,0.75,0.5,0.5,1,0.5,0.5,0.75,0.25,0.5,0.5,0.5,0.5,1,0.25,0.5,0.5,0.5
3,AND,1,0.75,0.75,1,1,0.75,1,1,0.5,0.75,0.75,0.5,0.25,1,0.5,0.75,0.25
4,ARE,1,0.25,0.75,0.75,0.5,0.25,0.75,0.25,0.75,0.75,0.75,0.25,0.25,0.25,1,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,VUT,0.25,0.25,0.25,0.5,0.25,0.25,0.25,0.75,0.25,0.5,0.75,0.25,1,0.25,0.25,0.25,0.5
182,WSM,0.75,0.25,0.25,1,0.25,0.5,0.75,0.5,0.25,0.25,0.75,0.25,1,0.5,0.25,0.5,0.5
183,ZAF,0.25,0.25,0.25,0.5,0.75,0.5,0.5,0.25,0.5,0.25,0.5,0.75,0.5,0.25,0.25,0.25,0.75
184,ZMB,0.25,0.25,0.25,0.5,0.5,0.25,0.25,0.25,0.25,0.25,0.25,1,1,0.25,0.25,0.25,0.5


In [ ]:
# Classify each SDG in 'low' or 'high' quantile
sdgs = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17']
for s in sdgs:
  m = y[s].mean()
  print(m)
  y[s] = y[s]-m
y

0.614247311827957
0.33736559139784944
0.43413978494623656
0.6438172043010753
0.4771505376344086
0.4435483870967742
0.6102150537634409
0.46236559139784944
0.4032258064516129
0.4529569892473118
0.5040322580645161
0.5604838709677419
0.6491935483870968
0.4153225806451613
0.3736559139784946
0.41935483870967744
0.4905913978494624


,iso3,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,AFG,-0.364247,-0.0873656,-0.18414,-0.393817,-0.227151,-0.193548,-0.110215,-0.212366,-0.153226,0.047043,-0.254032,0.189516,0.350806,-0.165323,-0.123656,-0.169355,-0.240591
1,AGO,-0.364247,-0.0873656,-0.18414,-0.393817,-0.227151,-0.193548,-0.360215,-0.212366,-0.153226,-0.202957,-0.254032,0.439516,0.100806,0.0846774,-0.123656,-0.169355,0.0094086
2,ALB,0.135753,0.162634,0.0658602,0.356183,0.0228495,0.0564516,0.139785,-0.212366,0.0967742,0.047043,-0.00403226,-0.0604839,0.350806,-0.165323,0.126344,0.0806452,0.0094086
3,AND,0.385753,0.412634,0.31586,0.356183,0.522849,0.306452,0.389785,0.537634,0.0967742,0.297043,0.245968,-0.0604839,-0.399194,0.584677,0.126344,0.330645,-0.240591
4,ARE,0.385753,-0.0873656,0.31586,0.106183,0.0228495,-0.193548,0.139785,-0.212366,0.346774,0.297043,0.245968,-0.310484,-0.399194,-0.165323,0.626344,0.0806452,0.0094086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,VUT,-0.364247,-0.0873656,-0.18414,-0.143817,-0.227151,-0.193548,-0.360215,0.287634,-0.153226,0.047043,0.245968,-0.310484,0.350806,-0.165323,-0.123656,-0.169355,0.0094086
182,WSM,0.135753,-0.0873656,-0.18414,0.356183,-0.227151,0.0564516,0.139785,0.0376344,-0.153226,-0.202957,0.245968,-0.310484,0.350806,0.0846774,-0.123656,0.0806452,0.0094086
183,ZAF,-0.364247,-0.0873656,-0.18414,-0.143817,0.272849,0.0564516,-0.110215,-0.212366,0.0967742,-0.202957,-0.00403226,0.189516,-0.149194,-0.165323,-0.123656,-0.169355,0.259409
184,ZMB,-0.364247,-0.0873656,-0.18414,-0.143817,0.0228495,-0.193548,-0.360215,-0.212366,-0.153226,-0.202957,-0.254032,0.439516,0.350806,-0.165323,-0.123656,-0.169355,0.0094086


In [ ]:
# Generate one-hot encoded vectors for SDG datasets
# Countries are classified on whether SDG index average if 'high' or 'low'
sdgs = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17']
y['mean'] = None
y['output'] = None
n = 0
for i in y.index.tolist():
  sdg = [0.,0.]
  m = np.mean(y[sdgs].loc[i])
  y['mean'].loc[i] = m
  if m<=0.:
    y['output'].loc[i] = [1.,0.]
    n = n+1
  else:
    y['output'].loc[i] = [0.,1.]
y

,iso3,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,mean,output
0,AFG,-0.364247,-0.0873656,-0.18414,-0.393817,-0.227151,-0.193548,-0.110215,-0.212366,-0.153226,0.047043,-0.254032,0.189516,0.350806,-0.165323,-0.123656,-0.169355,-0.240591,-0.134804,"[1.0, 0.0]"
1,AGO,-0.364247,-0.0873656,-0.18414,-0.393817,-0.227151,-0.193548,-0.360215,-0.212366,-0.153226,-0.202957,-0.254032,0.439516,0.100806,0.0846774,-0.123656,-0.169355,0.0094086,-0.134804,"[1.0, 0.0]"
2,ALB,0.135753,0.162634,0.0658602,0.356183,0.0228495,0.0564516,0.139785,-0.212366,0.0967742,0.047043,-0.00403226,-0.0604839,0.350806,-0.165323,0.126344,0.0806452,0.0094086,0.0710784,"[0.0, 1.0]"
3,AND,0.385753,0.412634,0.31586,0.356183,0.522849,0.306452,0.389785,0.537634,0.0967742,0.297043,0.245968,-0.0604839,-0.399194,0.584677,0.126344,0.330645,-0.240591,0.247549,"[0.0, 1.0]"
4,ARE,0.385753,-0.0873656,0.31586,0.106183,0.0228495,-0.193548,0.139785,-0.212366,0.346774,0.297043,0.245968,-0.310484,-0.399194,-0.165323,0.626344,0.0806452,0.0094086,0.0710784,"[0.0, 1.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,VUT,-0.364247,-0.0873656,-0.18414,-0.143817,-0.227151,-0.193548,-0.360215,0.287634,-0.153226,0.047043,0.245968,-0.310484,0.350806,-0.165323,-0.123656,-0.169355,0.0094086,-0.0906863,"[1.0, 0.0]"
182,WSM,0.135753,-0.0873656,-0.18414,0.356183,-0.227151,0.0564516,0.139785,0.0376344,-0.153226,-0.202957,0.245968,-0.310484,0.350806,0.0846774,-0.123656,0.0806452,0.0094086,0.0122549,"[0.0, 1.0]"
183,ZAF,-0.364247,-0.0873656,-0.18414,-0.143817,0.272849,0.0564516,-0.110215,-0.212366,0.0967742,-0.202957,-0.00403226,0.189516,-0.149194,-0.165323,-0.123656,-0.169355,0.259409,-0.0612745,"[1.0, 0.0]"
184,ZMB,-0.364247,-0.0873656,-0.18414,-0.143817,0.0228495,-0.193548,-0.360215,-0.212366,-0.153226,-0.202957,-0.254032,0.439516,0.350806,-0.165323,-0.123656,-0.169355,0.0094086,-0.105392,"[1.0, 0.0]"


In [ ]:
# Create dataset in dataframe form
dataframe = pd.concat([x[['iso3','input']],y[['output']]],axis=1)
dataframe

,iso3,input,output
0,AFG,"[0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, ...","[1.0, 0.0]"
1,AGO,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0]"
2,ALB,"[1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, ...","[0.0, 1.0]"
3,AND,"[0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, ...","[0.0, 1.0]"
4,ARE,"[0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...","[0.0, 1.0]"
...,...,...,...
181,VUT,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...","[1.0, 0.0]"
182,WSM,"[0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, ...","[0.0, 1.0]"
183,ZAF,"[0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, ...","[1.0, 0.0]"
184,ZMB,"[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[1.0, 0.0]"


In [ ]:
# Create one-hot encoded dataset in dict form
dataset = {}
dataset['input'] = np.vstack(dataframe['input'])
dataset['output'] = np.vstack(dataframe['output'])
dataset

{'input': array([[0., 0., 1., ..., 0., 1., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 1., 0.],
        ...,
        [0., 1., 0., ..., 1., 0., 0.],
        [1., 1., 1., ..., 0., 1., 1.],
        [0., 0., 0., ..., 0., 0., 0.]]), 'output': array([[1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 

In [ ]:
# Save dataset
dataframe.to_pickle(path+'data/dataframe.pkl')
pickle.dump(dataset,open(path+'data/dataset.pkl','wb'))

# (4) Train neural network model

In [ ]:
dataset = pickle.load(open(path+'data/dataset.pkl','rb'))

In [ ]:
dataset

{'input': array([[0., 0., 1., ..., 0., 1., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 1., 0.],
        ...,
        [0., 1., 0., ..., 1., 0., 0.],
        [1., 1., 1., ..., 0., 1., 1.],
        [0., 0., 0., ..., 0., 0., 0.]]), 'output': array([[1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 

In [ ]:
model = Sequential()
model.add(Dense(10,input_shape=(10,),activation='relu'))
model.add(Dense(50,activation='relu'))
model.add(Dense(100,activation='relu'))
model.add(Dense(50,activation='relu'))
model.add(Dense(2,activation='softmax'))
optimizer = tf.keras.optimizers.Adam(0.0005)
optimizer.learning_rate.assign(0.005)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=0.005>

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [ ]:
history = model.fit(dataset['input'],dataset['output'],epochs=300,batch_size=5)

Epoch 1/300
38/38 [==============================] - 1s 2ms/step - loss: 0.6852 - accuracy: 0.4624
Epoch 2/300
38/38 [==============================] - 0s 2ms/step - loss: 0.6021 - accuracy: 0.6935
Epoch 3/300
38/38 [==============================] - 0s 2ms/step - loss: 0.5461 - accuracy: 0.7419
Epoch 4/300
38/38 [==============================] - 0s 2ms/step - loss: 0.5245 - accuracy: 0.8118
Epoch 5/300
38/38 [==============================] - 0s 2ms/step - loss: 0.4634 - accuracy: 0.8118
Epoch 6/300
38/38 [==============================] - 0s 2ms/step - loss: 0.4562 - accuracy: 0.8011
Epoch 7/300
38/38 [==============================] - 0s 2ms/step - loss: 0.4481 - accuracy: 0.8065
Epoch 8/300
38/38 [==============================] - 0s 2ms/step - loss: 0.4252 - accuracy: 0.8387
Epoch 9/300
38/38 [==============================] - 0s 2ms/step - loss: 0.4187 - accuracy: 0.8118
Epoch 10/300
38/38 [==============================] - 0s 2ms/step - loss: 0.4143 - accuracy: 0.8172
Epoch 11/

In [ ]:
# ['fdi','dci','trsb','sis','ttcr','eif','bedi','dcps','slri','ans']
model.predict([[1.,0.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,1.,0.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,1.,0.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,1.,0.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,1.,0.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,1.,0.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,1.,0.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,1.,0.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,1.,0.],[0.,0.,0.,0.,0.,0.,0.,0.,0.,1.]])

array([[1.0000000e+00, 4.7132522e-35],
       [1.0000000e+00, 6.6451843e-26],
       [1.0000000e+00, 2.1811507e-10],
       [9.9999988e-01, 1.4067687e-07],
       [1.0000000e+00, 1.1691788e-11],
       [9.9938631e-01, 6.1361166e-04],
       [1.0000000e+00, 6.1682786e-15],
       [9.5980978e-01, 4.0190201e-02],
       [1.1118109e-06, 9.9999893e-01],
       [9.9999714e-01, 2.8554880e-06]], dtype=float32)

In [ ]:
model.save(path+'model/weights_1.0')

INFO:tensorflow:Assets written to: drive/My Drive/dev/sdi/model/weights_1.0/assets


# (5) Generate and evaluate results

In [ ]:
model = keras.models.load_model(path+'model/weights')

In [ ]:
def combinations(n,k):
  result = []
  for bits in itertools.combinations(range(n),k):
    s = [0.]*n
    for bit in bits:
      s[bit] = 1.
    result = result+[s]

  return result

In [ ]:
# Get results for all combinations of the inputs
datasets = ['fdi','dci','trsb','sis','ttcr','eif','bedi','dcps','slri','ans']
n = len(datasets)

inputs = [[0.]*n]
for k in range(n):
  inputs = inputs+combinations(n,k+1)

inputs

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,

In [ ]:
# Generate model predictions
sdg = model.predict(inputs)
sdg

array([[9.9991870e-01, 8.1259321e-05],
       [1.0000000e+00, 4.7132522e-35],
       [1.0000000e+00, 6.6451843e-26],
       ...,
       [2.3276722e-28, 1.0000000e+00],
       [6.4190054e-01, 3.5809949e-01],
       [3.3245203e-13, 1.0000000e+00]], dtype=float32)

In [ ]:
# Compute the cosine similarity of result vectors and best vector [0.,1.]
best = [0.,1.]
n = len(sdg)
cos = []
for i in range(n):
  cos = cos+[np.dot(sdg[i],best)/(np.linalg.norm(sdg[i])*np.linalg.norm(best))]
cos

[8.126592770258405e-05,
 4.713252219344022e-35,
 6.6451843066878e-26,
 2.1811506767388522e-10,
 1.406768913637293e-07,
 1.1691788162626615e-11,
 0.0006139883455360295,
 6.1682785750883165e-15,
 0.04183643128523676,
 1.0,
 2.8554963888518107e-06,
 1.3542878818384452e-08,
 3.898960443393662e-08,
 1.0,
 2.3708264240696053e-09,
 0.0,
 4.928313668364838e-31,
 2.029070689246737e-09,
 7.163445641741416e-36,
 1.8970909135112058e-19,
 1.0,
 1.9836727746525007e-11,
 2.445566065647328e-28,
 2.337198593576639e-23,
 1.3447411887258922e-19,
 1.7941727811648707e-17,
 1.0,
 0.9997077979321394,
 1.0,
 1.1558427635228678e-28,
 1.0,
 9.417481637172462e-25,
 1.0,
 0.0,
 3.052519378216566e-14,
 0.827932963576,
 0.9999959353148394,
 0.7322798270655708,
 1.0,
 1.0,
 1.0,
 6.762679452901946e-07,
 3.957807615991908e-23,
 1.0,
 8.220033184320622e-24,
 3.098075220358771e-19,
 1.1813264430527663e-14,
 1.0,
 1.0,
 2.3738874663653005e-07,
 1.0,
 9.359276695411131e-19,
 5.8447962025809375e-09,
 7.2092536319155645e-0

In [ ]:
a = {}
for i in range(len(cos)):
  a[i] = inputs[i]+[cos[i]]
a = pd.DataFrame(a).T
a.columns = ['fdi','dci','trsb','sis','ttcr','eif','bedi','dcps','slri','ans','cos']
a = a.sort_values(by=['cos'],ascending=False).reset_index(drop=True)
a

,fdi,dci,trsb,sis,ttcr,eif,bedi,dcps,slri,ans,cos
0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0
2,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
3,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
4,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1019,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
1020,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1021,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1022,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0


In [ ]:
k = a['cos']>=0.9
d = a[k]
d

,fdi,dci,trsb,sis,ttcr,eif,bedi,dcps,slri,ans,cos
0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.000000
1,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.000000
2,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.000000
3,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.000000
4,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
633,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.920255
634,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.915808
635,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.911955
636,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.903843


In [ ]:
b = d[['fdi','dci','trsb','sis','ttcr','eif','bedi','dcps','slri','ans','cos']].sum(axis=0)
c = b/len(d)
c = c.sort_values(ascending=False)
c

cos     0.997801
sis     0.667712
dcps    0.587774
eif     0.531348
trsb    0.531348
ans     0.525078
fdi     0.520376
ttcr    0.493730
slri    0.489028
dci     0.467085
bedi    0.459248
dtype: float64

In [ ]:
k = a['cos']<=0.1
d = a[k]
d

,fdi,dci,trsb,sis,ttcr,eif,bedi,dcps,slri,ans,cos
698,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.076191
699,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.061424
700,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.058100
701,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.049961
702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.041836
...,...,...,...,...,...,...,...,...,...,...,...
1019,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.000000
1020,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.000000
1021,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.000000
1022,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.000000


In [ ]:
b = d[['fdi','dci','trsb','sis','ttcr','eif','bedi','dcps','slri','ans','cos']].sum(axis=0)
c = b/len(d)
c = c.sort_values(ascending=False)
c

bedi    0.573620
slri    0.530675
dci     0.527607
ttcr    0.524540
fdi     0.457055
ans     0.432515
eif     0.417178
trsb    0.407975
dcps    0.325153
sis     0.180982
cos     0.001852
dtype: float64